In [ ]:
# Loading dataset, getting the new predictors

ATI = pd.read_csv('ATI.csv')

ATI = ATI[['revenue','imdb_score','budget','twitter_score','actors_score']]

# Getting new extended predictors

imdb2 = ATI[['imdb_score']] ** 2

twitter2 = ATI[['twitter_score']] ** 2

actors2 = ATI[['actors_score']] ** 2

budget2 = ATI[['budget']] ** 2


# 2 way interactions

imdb_twitter = ATI[['imdb_score']].to_numpy() * ATI[['twitter_score']].to_numpy()

imdb_budget = ATI[['imdb_score']].to_numpy() * ATI[['budget']].to_numpy()

imdb_actors = ATI[['imdb_score']].to_numpy() * ATI[['actors_score']].to_numpy()


budget_twitter = ATI[['budget']].to_numpy() * ATI[['twitter_score']].to_numpy()

budget_actors = ATI[['budget']].to_numpy() * ATI[['actors_score']].to_numpy()


twitter_actors = ATI[['twitter_score']].to_numpy() * ATI[['actors_score']].to_numpy()

# 3 way interactions

imdb_budget_twitter = ATI[['imdb_score']].to_numpy() * ATI[['twitter_score']].to_numpy() * ATI[['budget']].to_numpy()
actors_budget_twitter = ATI[['actors_score']].to_numpy() * ATI[['twitter_score']].to_numpy() * ATI[['budget']].to_numpy()
imdb_budget_actors = ATI[['imdb_score']].to_numpy() * ATI[['actors_score']].to_numpy() * ATI[['budget']].to_numpy()
imdb_twitter_actors = ATI[['imdb_score']].to_numpy() * ATI[['twitter_score']].to_numpy() * ATI[['actors_score']].to_numpy()

# 4 way interaction

imdb_budget_twitter_actors = ATI[['imdb_score']].to_numpy() * ATI[['actors_score']].to_numpy() * ATI[['budget']].to_numpy() * ATI[['twitter_score']].to_numpy()



# New dataset with 15 predictors now added in

ATI['imdb_twitter'] = imdb_twitter
ATI['imdb_budget'] = imdb_budget
ATI['imdb_actors'] = imdb_actors
ATI['budget_twitter'] = budget_twitter
ATI['budget_actors'] = budget_actors
ATI['twitter_actors'] = twitter_actors
ATI['imdb_budget_twitter'] = imdb_budget_twitter
ATI['actors_budget_twitter'] = actors_budget_twitter
ATI['imdb_budget_actors'] = imdb_budget_actors
ATI['imdb_twitter_actors'] = imdb_twitter_actors
ATI['imdb_budget_twitter_actors'] = imdb_budget_twitter_actors
ATI[['imdb2']] = imdb2
ATI[['twitter2']] = twitter2
ATI[['actors2']] = actors2
ATI[['budget2']] = budget2

In [ ]:
# Model split

from sklearn.model_selection import train_test_split

# getting the predictions

ATI_preds = ATI[['budget','twitter_score','actors_score','imdb_score','imdb_twitter','imdb_budget','imdb_actors','budget_twitter','budget_actors','twitter_actors','imdb_budget_twitter','actors_budget_twitter','imdb_budget_actors','imdb_twitter_actors','imdb_budget_twitter_actors','imdb2','actors2','budget2','twitter2']]

ATI_response = ATI[['revenue']]

# High Budget - Low Budget split

ATI_high = ATI[ATI['revenue'] > 1000000000]
ATI_low = ATI[ATI['revenue'] < 1000000000]

ATI_high_preds = ATI_high[['budget','twitter_score','actors_score','imdb_score','imdb_twitter','imdb_budget','imdb_actors','budget_twitter','budget_actors','twitter_actors','imdb_budget_twitter','actors_budget_twitter','imdb_budget_actors','imdb_twitter_actors','imdb_budget_twitter_actors','imdb2','actors2','budget2','twitter2']]
ATI_high_response = ATI_high[['revenue']]

ATI_low_preds = ATI_low[['budget','twitter_score','actors_score','imdb_score','imdb_twitter','imdb_budget','imdb_actors','budget_twitter','budget_actors','twitter_actors','imdb_budget_twitter','actors_budget_twitter','imdb_budget_actors','imdb_twitter_actors','imdb_budget_twitter_actors','imdb2','actors2','budget2','twitter2']]
ATI_low_response = ATI_low[['revenue']]



In [ ]:
# Forward Subset Selection

from sklearn.linear_model import LinearRegression


# initialising iterative variables

r_score = 0
r_new = 0
new_col = None

columns = [] # final list of columns

while (r_new >= r_score) and (len(columns) < len(ATI.columns)): 
    if reg != None:
        reg_old = reg # tracking previous model, useful for later on 

    # Also tracking previous datasets, useful for final testing

    if (X_train,X_test) != None:

        X_train_final, X_test_final = X_train, X_test
        
    new_pred = '' # new predictor that will improve R-score
    r_score = r_new # old r-score
    
    if new_col != None:
        columns.append(new_col)

    r_max = 0
    
    for i in ATI_preds.columns:

        if ((i in columns) == False): # making sure i is not a predictor

            columns_new = columns.copy()
        
            columns_new.append(i)
        
            ATI_i = ATI[columns_new] # new dataset with temp new column added
        
            ATI_response = ATI[['revenue']]

            # PreProcessing, source https://medium.com/@lomashbhuva/elastic-net-regression-the-ultimate-guide-to-combining-ridge-and-lasso-eec3395a0fa1

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(ATI_i)
            X_train, X_test, y_train, y_test = train_test_split(X_scaled, ATI_response, test_size=0.3, random_state=1774231)


            # fitting model
        
            reg = LinearRegression().fit(X_train,y_train)

            # testing it 

            r2 = reg.score(X_test,y_test)


            # if r2 better than max, make it new max
            if (r2 > r_max):
                new_col = i
                r_max = r2

    r_new = r_max # next best r score value


print(columns)
print(r_new)

print(mean_absolute_error(y_test,reg_old.predict(X_test_final)))
# ANOVA test

reg_old.coef_
        

In [ ]:
new_col, columns

In [ ]:
# ElasticNet implementation, reference https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

from sklearn.linear_model import ElasticNet

from sklearn.metrics import mean_absolute_error,r2_score

from sklearn.preprocessing import StandardScaler 

# Measuring shifts in model performance for high ratios


# HyperParameter selection

alphas = [0.001,0.01,0.1,1]

ell1 = [0.25,0.5,0.75]

# PreProcessing, source https://medium.com/@lomashbhuva/elastic-net-regression-the-ultimate-guide-to-combining-ridge-and-lasso-eec3395a0fa1

scaler = StandardScaler()
X_scaled = scaler.fit_transform(ATI_preds)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, ATI[['revenue']], test_size=0.3, random_state=1774231)


# initalising numpy array for hyperparameter results values

hyper_scores = [] # will have 12 rows as 3 ell1 vals, 4 alphas vals, so 12 combinations. 4 cols because adding in r_scores, mae vals

for i in alphas:
    
    for j in ell1:
        
        elasticmodel = ElasticNet(alpha = i, l1_ratio = j, random_state = 1774231)

        elasticmodel.fit(X_train,y_train)

        preds = elasticmodel.predict(X_test)

        # r_score

        r = r2_score(y_test, preds)

        # MAE

        mae = mean_absolute_error(y_test, preds)
        
        # modifying hyper-parameter database

        hyper_scores.append([i,j,r,mae])


# turning into numpy array

hyper_scores = np.array(hyper_scores)

# turning into dataframe

hyper_scores = pd.DataFrame(hyper_scores)

print(hyper_scores)

In [ ]:
elasticmodel.coef_

In [ ]:
# ElasticNet implementation, reference https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

from sklearn.linear_model import Lasso,Ridge

from sklearn.metrics import mean_absolute_error,r2_score

from sklearn.preprocessing import StandardScaler 

# HyperParameter selection

alphas = [0.001,0.01,0.1,1]


# PreProcessing, source https://medium.com/@lomashbhuva/elastic-net-regression-the-ultimate-guide-to-combining-ridge-and-lasso-eec3395a0fa1

scaler = StandardScaler()
X_scaled = scaler.fit_transform(ATI_preds)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, ATI_response, test_size=0.3, random_state=1774231)

## LASSO

# initalising hyperparameter results values

hyper_scores_lasso = [] # will have 4 rows alphas vals. 4 cols because adding in r_scores, mae vals

for i in alphas:
    
        
        lasso = Lasso(alpha = i, random_state = 1774231)

        lasso.fit(X_train,y_train)

        preds = lasso.predict(X_test)

        # r_score

        r = r2_score(y_test, preds)

        # MAE

        mae = mean_absolute_error(y_test, preds)
        
        # modifying hyper-parameter database

        hyper_scores_lasso.append([i,r,mae])


# turning into numpy array

hyper_scores_lasso = np.array(hyper_scores_lasso)

# turning into dataframe

hyper_scores_lasso = pd.DataFrame(hyper_scores_lasso)

## RIDGE

# initalising hyperparameter results values

hyper_scores_ridge = [] # will have 4 rows alphas vals. 4 cols because adding in r_scores, mae vals

for i in alphas:
    
        
        ridge = Ridge(alpha = i, random_state = 1774231)

        ridge.fit(X_train,y_train)

        preds = ridge.predict(X_test)

        # r_score

        r = r2_score(y_test, preds)

        # MAE

        mae = mean_absolute_error(y_test, preds)
        
        # modifying hyper-parameter database

        hyper_scores_ridge.append([i,r,mae])


# turning into numpy array

hyper_scores_ridge = np.array(hyper_scores_ridge)

# turning into dataframe

hyper_scores_ridge = pd.DataFrame(hyper_scores_ridge)





In [ ]:
lasso.coef_ # coefficients for lasso regression

In [ ]:
ridge.coef_  # coefficients for ridge regression

In [ ]:
hyper_scores_ridge  # metrics for ridge regression

In [ ]:
hyper_scores_lasso # metrics for lasso regression

In [ ]:
# reference https://www.geeksforgeeks.org/machine-learning/random-forest-regression-in-python/

# Random-Forest


import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder


# tree depth vals
mtry = [10,50,100]

# predictors to consider for each split 
min_n = [2,5,10,15]


final_results = []

for i in mtry:
    for j in min_n:
        
        x_train, x_test, y_train, y_test = train_test_split(ATI_preds, ATI_response, test_size=0.3, random_state=1774231)

        # radom forest model/fit on training set, splitting on MAE
        
        regressor = RandomForestRegressor(n_estimators=i, criterion = "absolute_error", min_samples_split = j, random_state=1774231)

        regressor.fit(x_train, y_train)


        # model testing on prediction
        predictions = regressor.predict(x_test)
        
        mae = mean_absolute_error(y_test, predictions)

        r2 = r2_score(y_test, predictions)

        final_results.append([i,j,r2,mae])



In [ ]:
final_results # seeing results for different r2, mae vals. 